In [44]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [45]:
# Load your dataset
data = pd.read_csv("./samletdata.csv", sep=";")

# Split data into features (X) and target variable (y)
X = data.drop(["price", "tax"], axis=1)
y = data['price']

# Define categorical features

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=999)

In [46]:
# Define categorical features
categorical_features = ["model", "transmission", "fuelType"]

# Create a pipeline for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [47]:
# Append linear regression model to pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scalar', StandardScaler(with_mean=False)),
    ('poly', PolynomialFeatures(degree=3)),
    ('regressor', LinearRegression())
], verbose=True)

# Train the model
model.fit(X_train, y_train)

[Pipeline] ...... (step 1 of 4) Processing preprocessor, total=   0.3s
[Pipeline] ............ (step 2 of 4) Processing scalar, total=   0.0s
[Pipeline] .............. (step 3 of 4) Processing poly, total=   0.3s
[Pipeline] ......... (step 4 of 4) Processing regressor, total= 5.4min


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['model', 'transmission',
                                                   'fuelType'])])),
                ('scalar', StandardScaler(with_mean=False)),
                ('poly', PolynomialFeatures(degree=3)),
                ('regressor', LinearRegression())],
         verbose=True)

In [48]:
# Score the model
score = model.score(X_test, y_test)

print("Model R-squared (R2) Score:", score)

Model R-squared (R2) Score: 0.6969896309554846


# testede $r^2$ på polynomisk lineært
- audi: 0.64
- bmw:  0.61
- ford: 0.46
- hyundai: 0.67
- mercedes: 0.40
- skoda: 0.78
- toyota: 0.79
- vauxhall: 0.45
- vw: 0.64

In [49]:
import matplotlib.pyplot as plt
from statistics import mean
plt.figure()
# Select a single row from X_test
single_row = X_test.iloc[[0]]  # Select the first row, adjust index as needed
print(single_row)

# Transform the single row to match the input format expected by the model
single_row_transformed = preprocessor.transform(single_row)

print(f"{y_test.iloc[0]}: {mean(model.predict(single_row))}")

       Unnamed: 0  model  year transmission  mileage fuelType   mpg  \
40380         966   Kona  2019       Manual     4180   Petrol  44,1   

      engineSize    brand  
40380        1,0  hyundai  
15990: 14566.13866866902


<Figure size 640x480 with 0 Axes>